In [1]:
import pandas as pd 
import numpy as np 

In [2]:
import data_clean_for_model
import PipelineHelper

/Users/benji/anaconda3/envs/play/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [8]:
import pickle

In [11]:
### 1. Load Data
df = pd.read_parquet("data/all_processed_df.parquet.gzip")

In [12]:
k = 5
rseed = 229
df["outcome"] = np.where( df["state"]=="successful", 1, 0, )
df["un_id"] = np.arange(0, df.shape[0], 1 )
df["name_len"] = df["name"].str.len()
df["cv_group"] = np.random.choice( np.arange(0, k), size=df.shape[0] )
df["binned_usd_goal"] = pd.qcut( np.log(df["usd_goal"]+1), 20 )

In [9]:
## load project metadata
try:
    f = open("data/features.pkl", "rb")
    ft_dict = pickle.load(f)
    f.close()
    X_train, y_train, X_test, y_test = ft_dict.values()
except:
    print("oops")

In [14]:
train_indices = X_train['un_id']

In [19]:
train_df = df[df.un_id.isin(train_indices)]

In [33]:
print(np.round(train_df[['outcome', 'usd_goal', 'blurb_len', 'name_len']].agg(['mean', 'min', 'max', 'median']), 2).to_latex())

\begin{tabular}{lrrrr}
\toprule
{} &  outcome &      usd\_goal &  blurb\_len &  name\_len \\
\midrule
mean   &      0.6 &  3.394373e+04 &     107.72 &     34.76 \\
min    &      0.0 &  1.000000e-02 &       1.00 &      1.00 \\
max    &      1.0 &  1.123396e+08 &     151.00 &     85.00 \\
median &      1.0 &  4.820280e+03 &     120.00 &     34.00 \\
\bottomrule
\end{tabular}



In [22]:
train_df[["currency", "country", "cat_id", "cat_parent_id", "loc_id", "loc_type"]].nunique()

currency            15
country             25
cat_id             171
cat_parent_id       15
loc_id           13771
loc_type             9
dtype: int64

In [35]:
for c in ["currency", "country", "cat_id", "cat_parent_id", "loc_id", "loc_type"]:
    print(c, 
         np.round(train_df.groupby(c).size().mean(), 2
         )
         )

currency 10324.87
country 6194.92
cat_id 905.69
cat_parent_id 9660.47
loc_id 11.24
loc_type 17190.89


In [36]:
for c in ["currency", "country", "cat_id", "cat_parent_id", "loc_id", "loc_type"]:
    print(c, 
         np.round(train_df.groupby(c).size().min(), 2
         )
         )

currency 14
country 13
cat_id 5
cat_parent_id 1218
loc_id 1
loc_type 1


In [37]:
for c in ["currency", "country", "cat_id", "cat_parent_id", "loc_id", "loc_type"]:
    print(c, 
         np.round(train_df.groupby(c).size().max(), 2
         )
         )

currency 106524
country 106524
cat_id 5280
cat_parent_id 19527
loc_id 6905
loc_type 143863
